# Section 7: Experiment Viewer（Streamlitアプリ）

## 🎯 このセクションの目標
- **Streamlit in Snowflake** で実験結果を閲覧できるアプリを作成
- 過去の実験のSHAP画像やメトリクスを簡単に確認できるようにする

## 📋 なぜ必要？
- Snowsight UIではアーティファクト画像を直接プレビューできない
- 過去の実験を振り返る際に、毎回ノートブックでコードを書くのは面倒
- → **ワンクリックで過去の実験を可視化**できるアプリがあると便利！

---

## 💡 完成イメージ

```
┌─────────────────────────────────────────────────────────┐
│  🔬 Experiment Viewer                                   │
├─────────────────────────────────────────────────────────┤
│  実験名: [CHURN_PREDICTION_EXPERIMENT ▼]                │
│  Run:    [Baseline ▼]                                   │
├─────────────────────────────────────────────────────────┤
│  📊 メトリクス                                          │
│  ┌────────┬────────┬────────┬────────┐                 │
│  │F1: 0.78│AUC:0.85│Acc:0.82│Time:2s │                 │
│  └────────┴────────┴────────┴────────┘                 │
├─────────────────────────────────────────────────────────┤
│  🖼️ アーティファクト                                    │
│  [Confusion Matrix] [Feature Importance] [SHAP Plot]   │
│                                                         │
│         (選択した画像がここに表示)                       │
└─────────────────────────────────────────────────────────┘
```


---
## 7.1 Streamlitアプリの作成手順

### Step 1: Streamlit App作成画面を開く

1. Snowsight左メニュー → **Projects** → **Streamlit**
2. 右上の **+ Streamlit App** をクリック

### Step 2: アプリ設定

| 項目 | 値 |
|------|---|
| App name | `EXPERIMENT_VIEWER` |
| App location (Database) | `MLOPS_HOL_DB` |
| App location (Schema) | `FEATURE_STORE` |
| App warehouse | `MLOPS_HOL_ML_WH` |

### Step 3: コードを貼り付け

下のセルのコードを **全て** コピーして、Streamlitエディタに貼り付けてください。


In [ ]:
# ============================================================
# 📋 以下のコードをコピーしてStreamlit in Snowflakeに貼り付け
# ============================================================

streamlit_code = '''
import streamlit as st
from snowflake.snowpark.context import get_active_session
from snowflake.ml.experiment import ExperimentTracking
import pandas as pd
import json
import os

# ページ設定
st.set_page_config(
    page_title="Experiment Viewer",
    page_icon="🔬",
    layout="wide"
)

# セッション取得
session = get_active_session()

# タイトル
st.title("🔬 Experiment Viewer")
st.markdown("過去の実験結果とアーティファクトを閲覧できます")

st.divider()

# ============================================================
# サイドバー: 実験・Run選択
# ============================================================
with st.sidebar:
    st.header("📂 実験の選択")
    
    # 実験結果テーブルからデータ取得
    @st.cache_data(ttl=60)
    def load_experiment_data():
        df = session.table("MLOPS_HOL_DB.FEATURE_STORE.EXPERIMENT_RESULTS").to_pandas()
        return df
    
    try:
        exp_df = load_experiment_data()
        
        # 実験名選択
        experiment_names = exp_df["EXPERIMENT_NAME"].unique().tolist()
        selected_experiment = st.selectbox(
            "実験名",
            experiment_names,
            index=0
        )
        
        # 選択した実験のRunをフィルタ
        filtered_df = exp_df[exp_df["EXPERIMENT_NAME"] == selected_experiment]
        run_names = filtered_df["RUN_NAME"].tolist()
        
        # Run選択
        selected_run = st.selectbox(
            "Run名",
            run_names,
            index=0
        )
        
        # 選択したRunのデータ
        run_data = filtered_df[filtered_df["RUN_NAME"] == selected_run].iloc[0]
        
        st.success(f"✅ {selected_run} を選択中")
        
    except Exception as e:
        st.error(f"データ取得エラー: {e}")
        st.info("Section 4を実行してEXPERIMENT_RESULTSテーブルを作成してください")
        st.stop()

# ============================================================
# メインエリア: メトリクス表示
# ============================================================
st.header("📊 メトリクス")

col1, col2, col3, col4, col5 = st.columns(5)

with col1:
    st.metric("F1 Score", f"{run_data['F1_SCORE']:.4f}")
with col2:
    st.metric("ROC-AUC", f"{run_data['ROC_AUC']:.4f}")
with col3:
    st.metric("Accuracy", f"{run_data['ACCURACY']:.4f}")
with col4:
    st.metric("Precision", f"{run_data['PRECISION']:.4f}")
with col5:
    st.metric("Recall", f"{run_data['RECALL']:.4f}")

# 追加メトリクス
col6, col7 = st.columns(2)
with col6:
    st.metric("学習時間", f"{run_data['TRAINING_TIME_SEC']:.2f} 秒")
with col7:
    cm = json.loads(run_data['CONFUSION_MATRIX'])
    st.metric("True Positives", f"{cm['tp']:,}")

st.divider()

# ============================================================
# パラメータ & 特徴量重要度
# ============================================================
col_left, col_right = st.columns(2)

with col_left:
    st.header("⚙️ パラメータ")
    params = json.loads(run_data['PARAMS'])
    st.json(params)

with col_right:
    st.header("📈 特徴量重要度")
    importance = json.loads(run_data['FEATURE_IMPORTANCE'])
    # 重要度順にソート
    sorted_importance = dict(sorted(importance.items(), key=lambda x: x[1], reverse=True))
    
    # 棒グラフで表示
    import altair as alt
    imp_df = pd.DataFrame({
        "Feature": list(sorted_importance.keys()),
        "Importance": list(sorted_importance.values())
    })
    chart = alt.Chart(imp_df).mark_bar().encode(
        x=alt.X("Importance:Q"),
        y=alt.Y("Feature:N", sort="-x"),
        color=alt.value("steelblue")
    ).properties(height=250)
    st.altair_chart(chart, use_container_width=True)

st.divider()

# ============================================================
# SHAP値
# ============================================================
st.header("🔍 SHAP値（平均絶対値）")

shap_importance = json.loads(run_data['SHAP_IMPORTANCE'])
sorted_shap = dict(sorted(shap_importance.items(), key=lambda x: x[1], reverse=True))

shap_df = pd.DataFrame({
    "Feature": list(sorted_shap.keys()),
    "SHAP Value": list(sorted_shap.values())
})
shap_chart = alt.Chart(shap_df).mark_bar().encode(
    x=alt.X("SHAP Value:Q"),
    y=alt.Y("Feature:N", sort="-x"),
    color=alt.value("coral")
).properties(height=250)
st.altair_chart(shap_chart, use_container_width=True)

st.divider()

# ============================================================
# アーティファクト画像の表示
# ============================================================
st.header("🖼️ アーティファクト画像")

# Experiment Tracking初期化
exp = ExperimentTracking(session=session)
exp.set_experiment(selected_experiment)

# アーティファクト一覧取得
try:
    # Run名からdisplay_nameを抽出（Baseline_20251218_xxx → Baseline）
    display_name = selected_run.split("_")[0]
    
    # 画像タイプ選択
    image_type = st.radio(
        "表示する画像",
        ["Confusion Matrix", "Feature Importance", "SHAP Summary Plot"],
        horizontal=True
    )
    
    # 対応するファイル名
    artifact_map = {
        "Confusion Matrix": f"confusion_matrix_{display_name}.png",
        "Feature Importance": f"feature_importance_{display_name}.png",
        "SHAP Summary Plot": f"shap_summary_{display_name}.png"
    }
    
    artifact_name = artifact_map[image_type]
    
    # ダウンロード先
    download_path = "/tmp/experiment_viewer/"
    os.makedirs(download_path, exist_ok=True)
    
    # ボタンで画像をロード
    if st.button(f"📥 {image_type} を表示", type="primary"):
        with st.spinner("画像を取得中..."):
            try:
                exp.download_artifacts(
                    run_name=selected_run,
                    artifact_path=artifact_name,
                    target_path=download_path
                )
                
                image_path = f"{download_path}{artifact_name}"
                
                if os.path.exists(image_path):
                    st.image(image_path, caption=f"{image_type} - {selected_run}")
                    st.success("✅ 画像を表示しました")
                else:
                    st.warning(f"画像ファイルが見つかりません: {artifact_name}")
                    
            except Exception as e:
                st.error(f"画像取得エラー: {e}")
                st.info("アーティファクトが存在しない可能性があります")

except Exception as e:
    st.error(f"アーティファクト取得エラー: {e}")

st.divider()

# ============================================================
# データセット情報
# ============================================================
with st.expander("📋 データセット情報"):
    dataset_info = json.loads(run_data['DATASET_INFO'])
    col1, col2, col3 = st.columns(3)
    with col1:
        st.metric("総サンプル数", f"{dataset_info['total_samples']:,}")
    with col2:
        st.metric("訓練データ", f"{dataset_info['train_samples']:,}")
    with col3:
        st.metric("テストデータ", f"{dataset_info['test_samples']:,}")
    
    st.write(f"**データセット名**: {dataset_info['dataset_name']}")
    st.write(f"**特徴量数**: {dataset_info['num_features']}")
    st.write(f"**訓練データのチャーン率**: {dataset_info['train_churn_rate']:.1%}")
    st.write(f"**テストデータのチャーン率**: {dataset_info['test_churn_rate']:.1%}")

# フッター
st.divider()
st.caption("🔬 MLOps Hands-on - Experiment Viewer | Powered by Streamlit in Snowflake")
'''

print("=" * 70)
print("📋 以下のコードをStreamlit in Snowflakeにコピー＆ペーストしてください")
print("=" * 70)
print(streamlit_code)


---
## 7.2 アプリの機能解説

### 📊 メトリクス表示
```python
# EXPERIMENT_RESULTSテーブルから取得
st.metric("F1 Score", f"{run_data['F1_SCORE']:.4f}")
```
- Section 4で保存したテーブルから直接読み込み
- リアルタイムで最新データを表示

### 📈 特徴量重要度グラフ
```python
# JSONをパースして棒グラフ化
importance = json.loads(run_data['FEATURE_IMPORTANCE'])
```
- Altairで動的なグラフを生成
- 重要度順にソートして表示

### 🖼️ アーティファクト画像
```python
# Experiment Trackingからダウンロード
exp.download_artifacts(
    run_name=selected_run,
    artifact_path=artifact_name,
    target_path=download_path
)
st.image(image_path)
```
- ボタンクリックでオンデマンドダウンロード
- 必要な画像だけ取得するので軽量


---
## ✅ Section 7 完了！

### 学んだこと
- **Streamlit in Snowflake** でアプリを作成
- Experiment Trackingのデータを可視化
- アーティファクト（画像）をアプリ内で表示

### 作成したもの
- Streamlit App: `EXPERIMENT_VIEWER`
  - 実験・Run選択機能
  - メトリクス表示（F1, AUC, Accuracy等）
  - 特徴量重要度グラフ
  - SHAP値グラフ
  - アーティファクト画像表示

### 💡 活用シーン
- **実験レビュー**: 過去の実験結果をチームで共有
- **モデル選定**: 複数モデルの比較検討
- **監査対応**: モデルの説明責任を果たす
- **ドキュメント**: スクリーンショットでレポート作成

---

## 🎉 ハンズオン完了！

お疲れさまでした！以下のMLOpsコンポーネントを体験しました：

| Section | 内容 |
|---------|------|
| 01 | 環境構築 |
| 02 | Feature Store |
| 03 | Model Training |
| 04 | Experiment Tracking |
| 05 | Model Registry |
| 06 | Batch Inference |
| 07 | Experiment Viewer (Streamlit) |

これらを組み合わせることで、**本番運用可能なMLOpsパイプライン**を構築できます！
